In [1]:
import os
import shutil
import datetime
import subprocess
import re
import multiprocessing
import time
import json
import datetime

In [2]:
def get_curr_time():
    return datetime.datetime.now().strftime("%d-%h-%y | %H:%M")

In [3]:
def _load_seq_from_file(file_name):
    with open(file_name) as f:
        seq = f.readlines()
    return "".join([l.strip() for l in seq])

In [4]:

SIMULATION_ROOT_DIR = "simulation"
SIMULATION_EXEC = os.path.join(SIMULATION_ROOT_DIR, "bin")
SIMULATION_LIBS = os.path.join(SIMULATION_ROOT_DIR, "libs")
SIMULATION_RUN_DIRS= os.path.join(SIMULATION_ROOT_DIR, "runs")


In [5]:
def create_run_dir_name(gene_name):
    return gene_name + "_" + datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")

In [6]:
os.environ["PATH"] += os.pathsep + os.path.join(os.getcwd(), SIMULATION_EXEC)
os.environ["PATH"]

'/home/ubuntu/anaconda3/envs/lammps/bin:/home/ubuntu/anaconda3/condabin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin:/home/ubuntu/CTP/simulation/bin'

In [7]:
def gen_sim_data(folder_name):
    seq = _load_seq_from_file(os.path.join(folder_name, "seq.txt"))
    seq_name = folder_name.split(os.sep)[-1]
    dirs = next(os.walk(os.path.join(folder_name, 'directCoexistence')))[1]
    dirs = [dir[4:] for dir in dirs]
    return {"gene_name": seq_name, "seq": seq, "temps": dirs}

In [8]:
def gen_sim_data_done(folder_name):
    seq = _load_seq_from_file(os.path.join(folder_name, "seq.txt"))
    seq_name = folder_name.split(os.sep)[-1]
    dirs = next(os.walk(folder_name))[1]
    dirs = [dir[4:] for dir in dirs]
    return {"gene_name": seq_name, "seq": seq, "temps": dirs}

In [9]:
def read_sim_data(seq_file='simulation/simulation_to_run.json'):
    with open(seq_file) as f:
        return json.load(f)

In [10]:
def run_sample_lmp(temp_dir_name, CPUs_per_process):
    f = open(os.path.join(temp_dir_name, "sample.log"), "w+")
    CPUs_per_process_arg = str(CPUs_per_process)
    subprocess.run(["mpiexec", "-np", CPUs_per_process_arg, "lmp", "-in", "sample.lmp"], stdout=f, cwd=temp_dir_name)
    f.close()

In [11]:
def run_simulation(gene_name, seq, temps,
                   CPUs,
                   CPUs_per_process, 
                   run_dirs=SIMULATION_RUN_DIRS,
                   simulation_dir=SIMULATION_ROOT_DIR,
                   delete_lammpstrj=True):
    
    if len(seq) < 40:
        CPUs_for_first_stages = 8
    else:
        CPUs_for_first_stages = 16
    
    processes =  CPUs // CPUs_per_process
    
    simulation_dir_name = create_run_dir_name(gene_name)
    simulation_dir_name = os.path.join(run_dirs, simulation_dir_name)
    print(simulation_dir_name)
    os.mkdir(simulation_dir_name)
   
    with open(os.path.join(simulation_dir_name, "seq.txt"), "w+") as f:
        for i, letter in enumerate(seq, start=1):
            if i < len(seq):
                f.write(letter+"\n")
            else:
                f.write(letter)
        f.truncate()
    shutil.copy(os.path.join(SIMULATION_LIBS, "seq2config.py"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_charges_one_letter.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_charges_three_letters.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_types_three_letters.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "aa_types_one_letter.txt"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "potentials.dat"), simulation_dir_name)

    shutil.copy(os.path.join(SIMULATION_LIBS, "inputSingleChain.lmp"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "compress.lmp"), simulation_dir_name)
    shutil.copy(os.path.join(SIMULATION_LIBS, "sample.lmp"), simulation_dir_name)
    
    
    
    subprocess.run(["python", "seq2config.py"], cwd=simulation_dir_name)
    os.rename(os.path.join(simulation_dir_name, "myconfig.dat"), os.path.join(simulation_dir_name, "initialChainConfig.dat"))

    # subprocess.run(["python", "seq2config.py"], cwd=simulation_dir_name)

    print("step 1: running input single chain")
    f = open(os.path.join(simulation_dir_name, "inputSingleChain.log"), "w+")
    subprocess.run(["mpiexec", "-np", str(CPUs_for_first_stages), "lmp", "-in", "inputSingleChain.lmp"], stdout=f, cwd=simulation_dir_name)
    f.close()
    os.rename(os.path.join(simulation_dir_name, "finalSingleChainStructure.dat"), os.path.join(simulation_dir_name, "singleChainConfig.dat"))

    print("step 2: running compress single chain")
    f = open(os.path.join(simulation_dir_name, "compress.log"), "w+")
    subprocess.run(["mpiexec", "-np", str(CPUs_for_first_stages), "lmp", "-in", "compress.lmp"], stdout=f, cwd=simulation_dir_name)
    f.close()
    os.rename(os.path.join(simulation_dir_name, "fusfinalStructureSlab.dat"), os.path.join(simulation_dir_name, "initialSlab.dat"))
    
    print("step 3: direct coexistence")
    with open(os.path.join(simulation_dir_name, "replicate.lammpstrj")) as f:
        replicate_data = f.readlines()
        data_to_copy = []
        for line in replicate_data[::-1]:
            if line == 'ITEM: ATOMS id mol type q xu yu zu\n':
                break
            else:
                data_to_copy.append(line)
        data_to_copy = [line for line in data_to_copy[::-1]]

    with open(os.path.join(simulation_dir_name, "initialSlab.dat")) as f:
        slab_data = f.read()

    data_to_swap = "".join(data_to_copy)

    slab_data = re.sub(r'(Atoms # full\n)([\n\s0-9-.e]*)', rf'\1\n{data_to_swap}', slab_data)
    slab_data = re.sub(r'Velocities([\n\s0-9-.e]*)', r'\n', slab_data)

    zlo_zhi = re.findall(r"\d.+\szlo zhi\n", slab_data)

    zlo, zhi = [float(v.strip()) for v in zlo_zhi[0].split(" ")[:2]]

    deltaZ = (zhi- zlo)*2

    zhi = zhi+deltaZ

    zlo = zlo - deltaZ

    zlo_zhi_r = f'\n{zlo} {zhi} zlo zhi'

    slab_data = re.sub(r'\n.*zlo zhi', zlo_zhi_r, slab_data)

    with open(os.path.join(simulation_dir_name, "initialSlab.dat"), 'w+') as f:
        f.write(slab_data)
    
   
    temp_buf = []
    for temp in sorted(temps):
        temp_dir_name = os.path.join(simulation_dir_name, str(temp))
        if not os.path.exists(temp_dir_name):
            os.mkdir(temp_dir_name)
        shutil.copy(os.path.join(SIMULATION_LIBS, "potentials.dat"), temp_dir_name)
        shutil.copy(os.path.join(SIMULATION_LIBS, "da.py"), temp_dir_name)
        shutil.copy(os.path.join(simulation_dir_name, "initialSlab.dat"), temp_dir_name)
        with open(os.path.join(SIMULATION_LIBS, "sample.lmp")) as f:
            sample_lmp = f.read()
        sample_lmp = re.sub(
           r"(variable temperature equal)(\s\d+)", 
           rf"\1 {temp}", 
           sample_lmp
        )
       
        with open(os.path.join(temp_dir_name, "sample.lmp"), "w+") as f:
           f.write(sample_lmp)

        temp_buf.append({"temp": temp, "dir": temp_dir_name})
        if len(temp_buf) == proccesses:
            print(f"running {temp_buf}")
            print(get_curr_time())
            start_time = time.time()
            with multiprocessing.Pool(len(temp_buf)) as pool:
                params = [(v["dir"],) for v in temp_buf]
                results = [pool.apply_async(run_sample_lmp, args = p + (CPUs_per_process,)) for p in params]
    
                for r in results:
                    r.get()
                    #print('\t', r.get())
            temp_buf = []
            print(f'execution time: {(time.time() - start_time) / (60*60)} hours')
            print(get_curr_time())
    
    if temp_buf:
        print(f"running {temp_buf}")
        print(f"remaining temp, CPUs_pp = {CPUs//len(temp_buf)}")
        print(get_curr_time())
        
        start_time = time.time()
        #with multiprocessing.Pool(CPUs_per_process) as pool:
        with multiprocessing.Pool(len(temp_buf)) as pool:
            params = [(v["dir"],) for v in temp_buf]
            results = [pool.apply_async(run_sample_lmp, args = p + (CPUs//len(temp_buf),)) for p in params]

            for r in results:
                r.get()
                # rint('\t', r.get())
        temp_buf = []
        print(f'execution time: {(time.time() - start_time) / (60*60)} hours')
        print(get_curr_time())


    if delete_lammpstrj:
        for temp_folder in next(os.walk(simulation_dir_name))[1]:
            for file in os.listdir(os.path.join(simulation_dir_name, temp_folder)):
                if file.endswith("lammpstrj"):
                    # print(os.path.join("simulation", "runs", r_folder, temp_folder, file))
                    os.remove(os.path.join(simulation_dir_name, temp_folder, file))
        for file in next(os.walk(simulation_dir_name))[2]:
            if file.endswith("lammpstrj"):
                # print(os.path.join("simulation", "runs", r_folder, temp_folder, file))
                os.remove(os.path.join(simulation_dir_name, file))
        
    try:    
        #Calculating ps densities
        print("calculating ps densities")
        if not os.path.exists(os.path.join(simulation_dir_name, "ps_densities")):
            os.mkdir(os.path.join(simulation_dir_name, "ps_densities"))
        
        shutil.copy(os.path.join(SIMULATION_LIBS, "ppd.py"), os.path.join(simulation_dir_name, "ps_densities"))
        for temp in next(os.walk(simulation_dir_name))[1]:
            if temp != "ps_densities":
                print(os.path.join(simulation_dir_name, str(temp)))
                subprocess.run(["python", "da.py", "densities_chunked2.dat"], cwd=os.path.join(simulation_dir_name, str(temp)))
                os.rename(os.path.join(simulation_dir_name, temp, "densities_chunked2.dat_concentrations.txt"),
                          os.path.join(simulation_dir_name, "ps_densities", f"densities_chunked_temp{temp}.txt"))
    
        subprocess.run(["python", "ppd.py"], cwd=os.path.join(simulation_dir_name, "ps_densities"))
    
    except Exception:
        pass
        
        # f = open(os.path.join(simulation_dir_name, "compress.log"), "w+")
        # subprocess.run(["mpiexec", "-np", f"CPUs_per_process", "lmp", "-in", "sample.lmp"], stdout=f, cwd=temp_dir_name)
        # f.close()


In [13]:
def get_completed_simulations():
    sim_dirs = next(os.walk(os.path.join("simulation", "runs")))[1]
    comp_sims = ["_".join(dir_name.split("_")[:-6]) for dir_name in sim_dirs if dir_name!='']
    return [sim for sim in comp_sims if sim]
simulations_completed = get_completed_simulations()
# simulations_completed

In [14]:
seq_to_run = read_sim_data()
# seq_to_run

In [16]:
julias_simulations = ["TDP", "TIA", "EWSR", "RBM", "FUS"]

In [ ]:
class JuliasExeption(Exception):
    pass

In [ ]:
### CPUs=96
CPUs_per_process = 16
proccesses = CPUs // CPUs_per_process 

seq_to_run = read_sim_data()

log_name = f'simulation_run_{datetime.datetime.now().strftime("%Y_%m_%d_%H_%m_%s")}.log'

with open(os.path.join("simulation", "logs", log_name), "w+") as f:
    print(get_curr_time())
    mesg = f"CPUs {CPUs}, proccesses {proccesses}, CPUs_per_process {CPUs_per_process}\n"
    print(mesg)
    f.writelines(mesg+"\n")
    
    # for seq_name in seq_to_run:
    for sim_number, seq_name in enumerate(sorted(seq_to_run, key=lambda x: len(seq_to_run[x]['seq']))):
        if seq_name in simulations_completed:
            print(f"Sequence was already calculated {seq_name}")
            continue
        
        try:
            for j in julias_simulations:
                if seq_name.lower().startswith(j.lower()):
                    print(f"Julias sequence {seq_name}")
                    raise JuliasExeption
        except JuliasExeption:
            continue
        
     
        mesg = f"running simulation for {seq_name}"
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
        
        start_time = time.time()
        
        sim_args = {"gene_name": seq_name,
                    "seq": seq_to_run[seq_name]['seq'],
                    "temps": seq_to_run[seq_name]['temps']}
          
        mesg = f"seq len {len(sim_args['seq'])}"
        print(mesg)
        f.writelines(mesg+"\n")
    
        mesg = f"total temps {len(sim_args['temps'])}"
        print(mesg)
        f.writelines(mesg+"\n")
        
        run_simulation(** sim_args | {"CPUs": CPUs, "CPUs_per_process": CPUs_per_process})
        # for i in range(111111111):
        #     pass
        mesg = f'execution time: {(time.time() - start_time) / (60*60)} hours'
        print(mesg)
        f.writelines(mesg+"\n")
        print(get_curr_time())
    
        mesg = f'\n'
        print(mesg)
        f.writelines(mesg+"\n")
        #break

#f.close()

17-Aug-24 | 13:03
CPUs 96, proccesses 6, CPUs_per_process 16

Sequence was already calculated VAPGVG
Sequence was already calculated VPGVG
Sequence was already calculated AVPGVG
Sequence was already calculated LGAPVG
Sequence was already calculated TVPGVG
Sequence was already calculated GRGDSPY
Sequence was already calculated MCDB_SYNE7_Q8GJM6_Wildtype
Sequence was already calculated RPAGYDS
Sequence was already calculated RPLGYDS
Sequence was already calculated CTDP1_Q9Y5B0_Wildtype
Sequence was already calculated SUPT6H(Q7KZ85)_Wildtype
Sequence was already calculated IRS2_Q9Y4H2_Wildtype
Sequence was already calculated GKGDSPYG
Sequence was already calculated GRGDAPYQ
Sequence was already calculated GRGDSPYQ
Sequence was already calculated IDS_P22304_Wildtype
Sequence was already calculated QYPSDGRG
Sequence was already calculated VPHSRNGG
Sequence was already calculated LORF1_-5S+5T
Sequence was already calculated LORF1_Q9UN81_Wildtype
Sequence was already calculated TEST_Q9Y6M0_Wi

/home/ubuntu/CTP/simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/250'}, {'temp': 270, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/270'}, {'temp': 290, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/290'}, {'temp': 300, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/300'}, {'temp': 307, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/307'}, {'temp': 314, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/314'}]
17-Aug-24 | 13:07
execution time: 5.5740972701046205 hours
17-Aug-24 | 18:41
running [{'temp': 321, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/321'}, {'temp': 328, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/328'}, {'temp': 335, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/335'}, {'temp': 342, 'dir': 'simulation/runs/KIF2A_-5K+5R_2024_08_17_13_0

  File "/home/ubuntu/CTP/simulation/runs/KIF2A_-5K+5R_2024_08_17_13_08_1723899826/300/da.py", line 22
    except ValueError:
    ^
IndentationError: expected an indented block
/home/ubuntu/CTP/simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/250'}, {'temp': 270, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/270'}, {'temp': 290, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/290'}, {'temp': 300, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/300'}, {'temp': 307, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/307'}, {'temp': 314, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/314'}]
18-Aug-24 | 02:43
execution time: 5.149643051558071 hours
18-Aug-24 | 07:52
running [{'temp': 321, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/321'}, {'temp': 328, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/328'}, {'temp': 335, 'dir': 'simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723

  File "/home/ubuntu/CTP/simulation/runs/KIF2A_O00139_Wildtype_2024_08_18_02_08_1723948799/300/da.py", line 22
    except ValueError:
    ^
IndentationError: expected an indented block
/home/ubuntu/CTP/simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/seq2config.py:67: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  typescharges.append([int(types[p,1]),float(charges[k,1])])


step 2: running compress single chain
step 3: direct coexistence
running [{'temp': 250, 'dir': 'simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/250'}, {'temp': 270, 'dir': 'simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/270'}, {'temp': 290, 'dir': 'simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/290'}, {'temp': 300, 'dir': 'simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/300'}, {'temp': 307, 'dir': 'simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/307'}, {'temp': 314, 'dir': 'simulation/runs/GKAP1_Q5VSY0_Wildtype_2024_08_18_15_08_1723995626/314'}]
18-Aug-24 | 15:45
